In [353]:
# default_exp movies_metadata

# Movies_Metadata

> API details.

In [1]:
#hide
from nbdev.showdoc import *

In [179]:
import numpy as np
import pandas as pd
import pandas_profiling
from IPython.display import HTML, Image
from sklearn import preprocessing
from iso_language_codes import language_dictionary
import os 
import datetime
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim import corpora, models
import nltk

In [14]:
dir_path = os.path.dirname(os.path.realpath(''))+'\\Repos\\'

In [13]:
dir_path+'\\Repos\\'

'C:\\Users\\hhapp'

In [349]:
meta_df = pd.read_csv(dir_path+'data/movies_metadata.csv')
meta_df.head()

C:\Users\hhapp\anaconda3\envs\movie_recommender\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [69]:
meta_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [211]:
profile = meta_df.profile_report(title='Profiling Movies Metadata')
profile.to_file(output_file=dir_path+"html\\profiler.html")

C:\Users\hhapp\anaconda3\envs\movie_recommender\lib\site-packages\pandas_profiling\visualisation\plot.py:154: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)


### Notes on features

- Discarding revenue and budget columns as they are mostly zeros
- Homepage is mostly missing
- Genres has some strange values
- ID and IMDB_ID has duplicates

In [347]:
#export
def clean_metadata(df):
    """Drops certain erroneous rows and duplicates"""
    df = df.drop([35587, 19730])
    df['id'] = pd.to_numeric(df.id, errors='coerce')
    df = df.dropna(subset=['id'])
    df['id'] = df['id'].astype(int)
    df = df.drop_duplicates(subset=['id'], keep=False)
    df = df.drop_duplicates()
    
    return df.reset_index()

In [348]:
meta_df = clean_metadata(meta_df)

In [215]:
#export
def get_adult_feature(df):
    """Cleans the adult column by turning it into boolean. Uses False as default value for others."""
    return df.adult.map({'True': True, 'False': False}).fillna(False).astype(bool)

In [216]:
#export
def get_genre_names(df):
    """Returns list of genre Names per movie. Has an override dictionary for correcting some genres."""
    
    overide = {'Aniplex':'Anime', 'BROSTA TV':'Anime', 'Carousel Productions':'pageants',
          'GoHands':'Anime', 'Mardock Scramble Production Committee':'Anime', 
           'Sentai Filmworks':'Anime'}
    
    return df.genres.apply(lambda x: [e['name'] if e['name'] not in overide else overide[e['name']] for e in eval(x)])

In [282]:
#export
def labelEncoder_genres(genres):
    """Create label encoder object from genres"""
    
    genre_names = np.unique(np.concatenate(genres.values).ravel())

    le = preprocessing.LabelBinarizer()
    vecs = le.fit_transform(genre_names.tolist())
    
    return le, vecs

In [283]:
le, genre_vecs = labelEncoder_genres(get_genre_names(meta_df))
le.classes_

array(['Action', 'Adventure', 'Animation', 'Anime', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Odyssey Media',
       'Pulser Productions', 'Rogue State', 'Romance', 'Science Fiction',
       'TV Movie', 'Telescene Film Group Productions', 'The Cartel',
       'Thriller', 'Vision View Entertainment', 'War', 'Western',
       'pageants'], dtype='<U32')

In [219]:
len(le.classes_)

21

In [220]:
#export
def get_original_language(df):
    """Returns movie original language. Checks if the code is correct."""
    accepted_ISO = language_dictionary().keys()
    return df.original_language.apply(lambda x: x if x in accepted_ISO else 'NA').astype(str)

In [341]:
get_original_language(meta_df).unique()

array(['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'NA', 'ar', 'es', 'ru',
       'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy',
       'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'mk', 'bo', 'ca',
       'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo', 'ka',
       'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv', 'ta',
       'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms', 'sq',
       'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo', 'ay', 'kn', 'eu',
       'ne', 'pa', 'ky', 'gl', 'uz', 'sm', 'mt', 'hy', 'iu', 'lb', 'si'],
      dtype=object)

In [326]:
#export
def get_numerical_features(df):
    """Returns numerical features in the dataset. Sets NA to 0.0."""
    return df[['vote_count', 'vote_average', 
                    'runtime', 'popularity']] \
            .apply(lambda x: pd.to_numeric(x, errors='coerce')) \
            .fillna(0.0)

In [223]:
get_numerical_features(meta_df).describe()

,vote_count,vote_average,runtime,popularity
count,45451.000000,45451.000000,45451.000000,45451.000000
mean,109.908583,5.617705,93.584190,2.921277
std,491.357459,1.924835,38.956086,6.005837
min,0.000000,0.000000,0.000000,0.000000
25%,3.000000,5.000000,85.000000,0.385830
50%,10.000000,6.000000,95.000000,1.127377
75%,34.000000,6.800000,107.000000,3.678588
max,14075.000000,10.000000,1256.000000,547.488298


In [224]:
#export
def get_datetime(df):
    """Returns movie datetime. Fills NA with 2 years ago. 2 Years is to avoid being considered recent, but also not too far ago."""
    return pd.to_datetime(pd.to_datetime(df['release_date'], errors='coerce') \
               .fillna((datetime.datetime.now() - datetime.timedelta(days=2*365)).date()))

In [225]:
get_datetime(meta_df)

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45461   2018-07-21
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45451, dtype: datetime64[ns]

In [270]:
#export
def get_datetime_decade_label(d):
    """Returns datetime decade label from datetime"""
    r = d.apply(lambda x: np.floor((datetime.datetime.now().year - (x.year)) / 10.0) if x.date().year > 1930 else 6)
    r = r.rename('decade_label')
    return r

In [272]:
get_datetime_recent(get_datetime(meta_df))

0        False
1        False
2        False
3        False
4        False
         ...  
45461    False
45462    False
45463    False
45464    False
45465    False
Name: released_recently, Length: 45466, dtype: bool

In [275]:
#export
def get_datetime_recent(d):
    """Returns datetime boolean for if it was released less than 2 years"""
    r = d.apply(lambda x: True if (datetime.datetime.now() - (x)).days < 2*365 else False)
    r = r.rename('released_recently')
    return r

### Processing movie overview

In [169]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hhapp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [170]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [229]:
#export
def encode_movie_overview(meta_df):
    """Returns a dictionary of NLP models to embed movie overview"""
    
    # Tokenize, remove stopwords, lemmatize and stem 
    processed = meta_df.overview.astype(str).apply(preprocess)
    
    # Number of times a word appears in the training set. Drop words fewer than 15
    dictionary = gensim.corpora.Dictionary(processed)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    
    # Bag of words 
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed]
    
    # TFIDF
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    
    # LDA model of 10 topics
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
    
    # Produce the topic distribution per document
    vectors = []
    for b in bow_corpus:
        n = np.zeros(10)
        topics = lda_model_tfidf[b]
        for t in topics:
            n[t[0]] = t[1]
        vectors.append(n)
        
    return {'vectors': vectors, 'dictionary':dictionary, 
            'tfidf':tfidf, 'lda':lda_model_tfidf}

In [208]:
encode_movie_overview(meta_df)

[array([0.        , 0.22061111, 0.        , 0.        , 0.75177294,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.65231436, 0.        , 0.10776801, 0.21990009, 0.        ]),
 array([0.        , 0.        , 0.        , 0.7639755 , 0.        ,
        0.20742713, 0.        , 0.        , 0.        , 0.        ]),
 array([0.29500514, 0.        , 0.        , 0.        , 0.33370093,
        0.20810975, 0.135879  , 0.        , 0.        , 0.        ]),
 array([0.        , 0.        , 0.        , 0.56981534, 0.        ,
        0.21059769, 0.        , 0.        , 0.        , 0.18623637]),
 array([0.        , 0.61083204, 0.        , 0.        , 0.        ,
        0.29648858, 0.        , 0.        , 0.        , 0.06465653]),
 array([0.        , 0.        , 0.        , 0.        , 0.61366194,
        0.32914951, 0.        , 0.        , 0.        , 0.        ]),
 array([0.        , 0.27919051, 0.

### Putting it All together

In [350]:
def get_movie_features(df):
    
    """Constructs fetures for movies using functions constructed in this module"""
    
    df = clean_metadata(df)
    
    dates = get_datetime(df)
    
    genres = get_genre_names(df)
    
    le, genre_vecs = labelEncoder_genres(genres)
    
    genre_df = pd.DataFrame.from_records(genres.apply(lambda x: np.sum(le.transform(x), axis=0) if len(x) > 0 else np.zeros(len(le.classes_))).values)
    genre_df.columns = ['G_'+str(i) for i in range(len(genre_df.columns))]
    
    df_features=  pd.concat([df['id'].astype(int),
                            get_adult_feature(df),
                            get_numerical_features(df),
                            get_datetime_decade_label(dates),
                            get_datetime_recent(dates)], axis=1)
    
    
    
    return pd.concat([df_features, genre_df], axis=1).set_index('id')

In [351]:
get_movie_features(meta_df)

,adult,vote_count,vote_average,runtime,popularity,decade_label,released_recently,G_0,G_1,G_2,...,G_10,G_11,G_12,G_13,G_14,G_15,G_16,G_17,G_18,G_19
id,,,,,,,,,,,,,,,,,,,,,
862,False,5415.0,7.7,81.0,21.946943,2.0,False,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8844,False,2413.0,6.9,104.0,17.015539,2.0,False,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15602,False,92.0,6.5,101.0,11.712900,2.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
31357,False,34.0,6.1,127.0,3.859495,2.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11862,False,173.0,5.7,106.0,8.387519,2.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439050,False,1.0,4.0,90.0,0.072051,0.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111109,False,3.0,9.0,360.0,0.178241,0.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67758,False,6.0,3.8,90.0,0.903007,1.0,False,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
